In [3]:
import matplotlib
%matplotlib inline
from collections import deque

## Check Function

In [88]:
def test_wheather_the_same(solution_a, solution_b):
    '''
    params:solution_a, solution_b: lists
    check if the two permutation solutions for tour are actually the same
    '''
    dequeue_a = deque(solution_a)
    dequeue_b = deque(solution_b)
    while(dequeue_a[0]!=dequeue_b[0]):
        # circular shift from left to right
        dequeue_a.rotate()
    # check from left to right
    if dequeue_a == dequeue_b:
        return True
    else:
        dequeue_a.reverse()
        dequeue_a.rotate()
        if dequeue_a == dequeue_b:
            return True
        else:
            return False

def count_neighbors(neighbors):
    '''
    params: neighbors: a list of solutions probably with duplications
    output: actual number of neighbors
    '''
    total_count = len(neighbors) - 1
    for idx, solution_a in enumerate(neighbors):
        for solution_b in neighbors[idx+1:]:
            
            if test_wheather_the_same(solution_a, solution_b):
                print(solution_a, solution_b)
                print("same")
                total_count -= 1
                break
    return total_count

## Adjacent Swap

In [89]:
def ad_swap(var_size):
    init_solution = list(range(var_size))
    neighbors = list()
    neighbors.append(init_solution)
    for idx in range(var_size):
        newsolutoin = list(init_solution)
        tmp = newsolutoin[idx-1]
        newsolutoin[idx-1] = newsolutoin[idx]
        newsolutoin[idx] = tmp
        neighbors.append(newsolutoin)
    return neighbors

In [78]:
count_neighbors(ad_swap(4))

2

## Arbitary Swap

In [86]:
def arbitary_swap(var_size):
    init_solution = list(range(var_size))
    neighbors = list()
    neighbors.append(init_solution)
    for idx1 in range(var_size):
        for idx2 in range(idx1 + 1, var_size):
            newsolutoin = list(init_solution)
            tmp = newsolutoin[idx1]
            newsolutoin[idx1] = newsolutoin[idx2]
            newsolutoin[idx2] = tmp
            neighbors.append(newsolutoin)
    return neighbors

n >= 6 size = nC2

In [116]:
count_neighbors(arbitary_swap(40))

780

## Insertion

In [130]:
def insertion(var_size):
    init_solution = list(range(var_size))
    neighbors = list()
    neighbors.append(init_solution)
    for idx1 in range(var_size):
        for idx2 in range(var_size):
            newsolutoin = list(init_solution)
            del newsolutoin[idx1]
            if idx2 < idx1:
                newsolutoin.insert(idx2, idx1)
            elif idx2 > idx1:
                newsolutoin.insert(idx2, idx1)
            else:
                continue
            neighbors.append(newsolutoin)
    print(neighbors)
    return neighbors

In [141]:
count_neighbors(insertion(4))

[[0, 1, 2, 3], [1, 0, 2, 3], [1, 2, 0, 3], [1, 2, 3, 0], [1, 0, 2, 3], [0, 2, 1, 3], [0, 2, 3, 1], [2, 0, 1, 3], [0, 2, 1, 3], [0, 1, 3, 2], [3, 0, 1, 2], [0, 3, 1, 2], [0, 1, 3, 2]]
[0, 1, 2, 3] [1, 2, 3, 0]
same
[1, 0, 2, 3] [1, 0, 2, 3]
same
[1, 2, 0, 3] [0, 2, 1, 3]
same
[1, 2, 3, 0] [3, 0, 1, 2]
same
[1, 0, 2, 3] [0, 2, 3, 1]
same
[0, 2, 1, 3] [0, 2, 1, 3]
same
[0, 2, 3, 1] [2, 0, 1, 3]
same
[2, 0, 1, 3] [0, 1, 3, 2]
same
[0, 2, 1, 3] [0, 3, 1, 2]
same
[0, 1, 3, 2] [0, 1, 3, 2]
same


2

## Two-edge Inversion

In [117]:
def two_opt(var_size):
    init_solution = list(range(var_size))
    neighbors = list()
    neighbors.append(init_solution)
    for idx1 in range(var_size):
        for idx2 in range(var_size):
            newsolutoin = list(init_solution)
            del newsolutoin[idx1]
            if idx2 < idx1:
                newsolutoin.insert(idx2, idx1)
            elif idx2 > idx1:
                newsolutoin.insert(idx2, idx1)
            else:
                continue
            neighbors.append(newsolutoin)
    print(neighbors)
    return neighbors

In [118]:
del x[1]

In [119]:
x

[1, 3]